In [1]:
# imports
# from transformers import pipeline
from helpers import Video_loader, Image_loader, Captioner, Audio_transcriber, Fusion_helper
import tensorflow as tf

In [2]:
VIDEOS_DIRECTORY = "videos"
FRAME_FREQUENCY = 300

In [ ]:
# video captions section
video_loader = Video_loader(VIDEOS_DIRECTORY, FRAME_FREQUENCY)

captions_dict = video_loader.process()
# need to tune this up but for now it's whatever - we have frame visual captions now

print("------------------------------------------------------------------------------------------\n END OF HELPERS \n")
print(captions_dict)

In [6]:
import json

vid_cap_f = open("video_captions_dict_new.txt", 'w')
json.dump(captions_dict, vid_cap_f)


In [3]:
# audio captions section
audio_transcriber = Audio_transcriber(VIDEOS_DIRECTORY)
audio_captions_dict = audio_transcriber.transcribe_video_folder(VIDEOS_DIRECTORY)

Transcribing video folder: videos
Filenames: 
[<DirEntry '$250,000 car gets windshield SMASHED by kid on a skateboard!!! [l7yt-VPYtOA].mp4'>, <DirEntry "(Live and Breaking) Fire at Bangkok Nightclub - New Year's Eve [PrZ56-47TWg].mp4">, <DirEntry '14-year-old girl thrown off ride at amusement park in Chongqing [zCliWb4rdsY].mp4'>, <DirEntry '2011⧸3⧸11 東日本大震災 発生時(仙台市)  Earthquake in Sendai [j3fUqdGXLbM].mp4'>, <DirEntry '2018-Bush cracks joke to Obama while Clinton makes a speech. Full video Link is in description. [PUqtu1kIfKU].mp4'>, <DirEntry '25 wild dogs against a cyclist [6IuFoxpPNAM].mp4'>, <DirEntry '30 Dolphins stranding and incredibly saved! Extremely rare event! [ekmMD8oYtJ0].mp4'>, <DirEntry '5 Real Aliens Caught on Camera ● Real Proof Of Aliens Exist [vUnI0J3BYnI].mp4'>, <DirEntry 'Aana idayal Thrissur pooram 2012  tcv.mpg [vvU2h4-rgsw].mp4'>, <DirEntry 'Adele Medley - Casual Cover - Ronan Scolard & Glenn Murphy Live in Dublin [5RBcIPuKDUc].mp4'>, <DirEntry 'Alleged Boko Ha

In [6]:
print(audio_captions_dict)

{<DirEntry '$250,000 car gets windshield SMASHED by kid on a skateboard!!! [l7yt-VPYtOA].mp4'>: ' sorry Martin Shields milesplit', <DirEntry "(Live and Breaking) Fire at Bangkok Nightclub - New Year's Eve [PrZ56-47TWg].mp4">: '', <DirEntry '14-year-old girl thrown off ride at amusement park in Chongqing [zCliWb4rdsY].mp4'>: '', <DirEntry '2011⧸3⧸11 東日本大震災 発生時(仙台市)  Earthquake in Sendai [j3fUqdGXLbM].mp4'>: " Brother Bear teriyaki bento you don't go over there after effects of sex what is 70 ml planet audio stereo Elsa Pataky Terence Timothy soccer", <DirEntry '2018-Bush cracks joke to Obama while Clinton makes a speech. Full video Link is in description. [PUqtu1kIfKU].mp4'>: '', <DirEntry '25 wild dogs against a cyclist [6IuFoxpPNAM].mp4'>: " yesterday yesterday Dora Dora Dora benefit Thomas Donna Donna Donna Donna roller bender daughter I'm never gonna go to yoga Miller Subaru top 10 top of the ladder Natasha Natasha when was the Holocaust the sports academy live on Lobos daughter log

In [7]:
new_dict = dict()

In [8]:
for key in audio_captions_dict.keys():
    new_dict_key = key.name
    new_dict[new_dict_key] = audio_captions_dict[key]

print(new_dict)

{'$250,000 car gets windshield SMASHED by kid on a skateboard!!! [l7yt-VPYtOA].mp4': ' sorry Martin Shields milesplit', "(Live and Breaking) Fire at Bangkok Nightclub - New Year's Eve [PrZ56-47TWg].mp4": '', '14-year-old girl thrown off ride at amusement park in Chongqing [zCliWb4rdsY].mp4': '', '2011⧸3⧸11 東日本大震災 発生時(仙台市)  Earthquake in Sendai [j3fUqdGXLbM].mp4': " Brother Bear teriyaki bento you don't go over there after effects of sex what is 70 ml planet audio stereo Elsa Pataky Terence Timothy soccer", '2018-Bush cracks joke to Obama while Clinton makes a speech. Full video Link is in description. [PUqtu1kIfKU].mp4': '', '25 wild dogs against a cyclist [6IuFoxpPNAM].mp4': " yesterday yesterday Dora Dora Dora benefit Thomas Donna Donna Donna Donna roller bender daughter I'm never gonna go to yoga Miller Subaru top 10 top of the ladder Natasha Natasha when was the Holocaust the sports academy live on Lobos daughter logos meaning napa-sonoma tata Tata I'm gonna gonna take a little Tho

In [9]:
import json

aud_cap_f = open("audio_captions_dict", 'w')
json.dump(new_dict, aud_cap_f)

In [10]:
aud_cap_f.close()

In [ ]:
# explore dicts
print("---FRAME CAPTIONS---")
print(captions_dict)
print("\n\n---AUDIO CAPTIONS---")
print(audio_captions_dict)

# Continue from here to avoid repeating preprocessing

In [1]:
import json

In [2]:
def remove_trailing_information(seperator, string) -> str:
    result = string.partition(seperator)[0]
    result = result[0:-1]
    return result

In [3]:
def remove_leading_information(string) -> str:
    result = string.partition('\\')[2]
    return result


In [4]:
print(remove_trailing_information('!', 'Hello World! 123'))
print(remove_leading_information("videos\\\\$250,000 car gets windshield SMASHED by kid on a skateboard!!!"))

Hello Worl
\$250,000 car gets windshield SMASHED by kid on a skateboard!!!


In [5]:
f = open("video_captions_dict.txt")
captions_dict = json.load(f)
print(captions_dict)
f.close()

{'videos\\$250,000 car gets windshield SMASHED by kid on a skateboard!!! [l7yt-VPYtOA].mp4_images': [[{'generated_text': 'a crowd of people walking down a street'}], [{'generated_text': 'a man is driving a orange lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb'}], [{'generated_text': 'a person riding a skateboard down a street'}], [{'generated_text': 'a street with a sign and a car parked on the side'}], [{'generated_text': 'a man is standing in front of a car'}], [{'generated_text': 'a man in an orange car driving down the street'}]], "videos\\(Live and Breaking) Fire at Bangkok Nightclub - New Year's Eve [PrZ56-47TWg].mp4_images": [[{'generated_text': 'a group of people standing around a man'}], [{'generated_text': 'a man is holding a cell phone'}], [{'generated_text': 'a man in a white shirt is holding a knife'}], [{'generated_text': 'a fire is seen in the sky above a building'}], [{'generated_text': 'a man is standing in the dark'}], [{'generated_text': 'a fire is 

In [6]:
f = open("audio_captions_dict.txt")
audio_captions_dict = json.load(f)
audio_captions_dict_cleaned = {}
f.close()

for key in audio_captions_dict:
    new_key = remove_trailing_information('[', key)
    audio_captions_dict_cleaned[new_key] = audio_captions_dict[key]
print(audio_captions_dict_cleaned)

audio_captions_dict = audio_captions_dict_cleaned


{'$250,000 car gets windshield SMASHED by kid on a skateboard!!!': ' sorry Martin Shields milesplit', "(Live and Breaking) Fire at Bangkok Nightclub - New Year's Eve": '', '14-year-old girl thrown off ride at amusement park in Chongqing': '', '2011⧸3⧸11 東日本大震災 発生時(仙台市)  Earthquake in Sendai': " Brother Bear teriyaki bento you don't go over there after effects of sex what is 70 ml planet audio stereo Elsa Pataky Terence Timothy soccer", '2018-Bush cracks joke to Obama while Clinton makes a speech. Full video Link is in description.': '', '25 wild dogs against a cyclist': " yesterday yesterday Dora Dora Dora benefit Thomas Donna Donna Donna Donna roller bender daughter I'm never gonna go to yoga Miller Subaru top 10 top of the ladder Natasha Natasha when was the Holocaust the sports academy live on Lobos daughter logos meaning napa-sonoma tata Tata I'm gonna gonna take a little Thora Birch rewind 1 minute lotto tow truck Little Mermaid story Pretoria weather colour Bob Saget the box in m

In [7]:
# generate new frame caption dict with cleaned data
video_captions_dict = {}
for video_name in captions_dict.keys():
    combined_text = ""
    for top_level in captions_dict[video_name]:
        inside_dict = top_level[0]
        combined_text = combined_text + ' ' + inside_dict["generated_text"]
    trimmed_name = remove_trailing_information('[', video_name)
    trimmed_name = remove_leading_information(trimmed_name)
    video_captions_dict[trimmed_name] = combined_text

print(video_captions_dict)

{'$250,000 car gets windshield SMASHED by kid on a skateboard!!!': ' a crowd of people walking down a street a man is driving a orange lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb a person riding a skateboard down a street a street with a sign and a car parked on the side a man is standing in front of a car a man in an orange car driving down the street', "(Live and Breaking) Fire at Bangkok Nightclub - New Year's Eve": ' a group of people standing around a man a man is holding a cell phone a man in a white shirt is holding a knife a fire is seen in the sky above a building a man is standing in the dark a fire is seen in the sky above a building a car is seen in the dark at night a group of people walking down a street at night a woman is standing in the street with her arms outstretched a man is standing in the middle of a room a tornado is seen in this image from the video a man is holding a knife in his hand a man is seen in the dark as he walks through a crowd a

In [8]:
vid_phraselist : list = list(video_captions_dict.values())
print(vid_phraselist)

vid_wordlist : list = []
for phrase in vid_phraselist:
    vid_wordlist.extend(str.split(phrase))

print(vid_wordlist)

[' a crowd of people walking down a street a man is driving a orange lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb a person riding a skateboard down a street a street with a sign and a car parked on the side a man is standing in front of a car a man in an orange car driving down the street', ' a group of people standing around a man a man is holding a cell phone a man in a white shirt is holding a knife a fire is seen in the sky above a building a man is standing in the dark a fire is seen in the sky above a building a car is seen in the dark at night a group of people walking down a street at night a woman is standing in the street with her arms outstretched a man is standing in the middle of a room a tornado is seen in this image from the video a man is holding a knife in his hand a man is seen in the dark as he walks through a crowd a man in a suit and tie is standing on a stage a bunch of cars are parked in the street a car is shown in the dark', ' a black backgr

In [9]:
aud_phraselist : list = list(audio_captions_dict.values())
print(aud_phraselist)

aud_wordlist : list = []
for phrase in aud_phraselist:
    aud_wordlist.extend(str.split(phrase))

print(aud_wordlist)

[' sorry Martin Shields milesplit', '', '', " Brother Bear teriyaki bento you don't go over there after effects of sex what is 70 ml planet audio stereo Elsa Pataky Terence Timothy soccer", '', " yesterday yesterday Dora Dora Dora benefit Thomas Donna Donna Donna Donna roller bender daughter I'm never gonna go to yoga Miller Subaru top 10 top of the ladder Natasha Natasha when was the Holocaust the sports academy live on Lobos daughter logos meaning napa-sonoma tata Tata I'm gonna gonna take a little Thora Birch rewind 1 minute lotto tow truck Little Mermaid story Pretoria weather colour Bob Saget the box in my Subaru we're good", '', ' satish finder.com.au', ' 100 100', ' you are a boy', " new London what's 9 Russian Anand sheela marijuana love you too I'm talking about things that you know may 11 February one minute later how do I learn more vocabulary why do animals hibernate yeah are you letting you know I'm on my a part of our TV what are coming sooner in the moon yeah I know Hann

In [10]:
titles_raw = list(video_captions_dict.keys())
title_wordlist : list = []

In [11]:
for title in titles_raw:
    cleaned_title = title[8:]
    title_wordlist.extend(str.split(cleaned_title))

print(title_wordlist)

['car', 'gets', 'windshield', 'SMASHED', 'by', 'kid', 'on', 'a', 'skateboard!!!', 'd', 'Breaking)', 'Fire', 'at', 'Bangkok', 'Nightclub', '-', 'New', "Year's", 'Eve', 'old', 'girl', 'thrown', 'off', 'ride', 'at', 'amusement', 'park', 'in', 'Chongqing', '1', '東日本大震災', '発生時(仙台市)', 'Earthquake', 'in', 'Sendai', 'h', 'cracks', 'joke', 'to', 'Obama', 'while', 'Clinton', 'makes', 'a', 'speech.', 'Full', 'video', 'Link', 'is', 'in', 'description.', 'dogs', 'against', 'a', 'cyclist', 'ins', 'stranding', 'and', 'incredibly', 'saved!', 'Extremely', 'rare', 'event!', 'liens', 'Caught', 'on', 'Camera', '●', 'Real', 'Proof', 'Of', 'Aliens', 'Exist', 'yal', 'Thrissur', 'pooram', '2012', 'tcv.mpg', 'dley', '-', 'Casual', 'Cover', '-', 'Ronan', 'Scolard', '&', 'Glenn', 'Murphy', 'Live', 'in', 'Dublin', 'Boko', 'Haram', 'Video', 'Depicting', 'Leader', 'Shekau', 'Surrendering', 'Airline', 'plane', 'catches', 'fire', 'on', 'runway', 'Ninja', 'Warrior：', 'Old', 'Man', 'Run', 'Sniper', 'Phonecall', 'scene'

In [12]:
complete_wordlist : list = []
complete_wordlist.extend(title_wordlist)
complete_wordlist.extend(aud_wordlist)
complete_wordlist.extend(vid_wordlist)

In [13]:
import numpy as np
def load_glove_vectors(glove_file):
    embeddings = {}
    with open(glove_file, 'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

glove_file = 'glove.6B.50d.txt'
glove_vectors = load_glove_vectors(glove_file)

In [14]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(complete_wordlist)

In [15]:
embedding_matrix = np.zeros((len(complete_wordlist), 50))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_vectors.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [16]:
from keras.layers import Embedding
from keras.initializers import Constant

embedding_layer = Embedding(
    input_dim=len(complete_wordlist), 
    output_dim=50, 
    embeddings_initializer=Constant(embedding_matrix), 
    trainable=False
)

In [17]:
# model itself:
import tensorflow as tf
model = tf.keras.Sequential([
    embedding_layer,
    # add bidirectional element to LSTM, maybe even convert to GRU
    tf.keras.layers.LSTM(units=64),
    tf.keras.layers.Dense(1)
])

In [18]:
# model compilation
# steps to achieve:
# - implement transformer model to analyse all text
# - decide on loss - binary cross-entropy most likely
# - decide on optimiser
# - compile model
model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=['accuracy']
)

In [19]:
# training
# steps to achieve:
# - download all data
# - run preprocessing on all of the videos

In [20]:
# need to get fake/real status of each video
title_dict = None
with open("title_dict.txt", 'r') as f:
    title_dict = json.load(f)
    f.close()
print(title_dict)

{'https://www.youtube.com/watch?v=Y7MJlmbnLQA': 'بنت سقطت في حمم بركانيه 🙀', 'https://www.youtube.com/watch?v=AEPvSo8bE2I': 'Michel & Sven - Der Tischdeckentrick - Teil 4 / The Table Cloth Magic Trick (Original)', 'https://www.youtube.com/watch?v=CXOU10WBogs': 'Backflip and almost hit by car!', 'https://www.youtube.com/watch?v=ZoHAf7VtIhA': 'Shark Attack - Cam Girl Bitten off Florida Coast', 'https://www.youtube.com/watch?v=RXF5yZZe8nQ': 'American Airline plane catches fire on runway', 'https://www.youtube.com/watch?v=fSbssO2DR7A': 'One Wing Landing', 'https://www.youtube.com/watch?v=PtWNknwUCE8': 'Reporter hit by stop sign during live tv.', 'https://www.youtube.com/watch?v=2YTntpldkhw': 'Kid from make a wish foundation gets tackled', 'https://www.youtube.com/watch?v=0NVUSOm8Mps': 'The Wrath of an Inflatable Raft', 'https://www.youtube.com/watch?v=F4jjR4vkwkM': 'Ski Trick Fail', 'https://www.youtube.com/watch?v=jGuDpD-Y-1s': 'girl falls through window', 'https://www.youtube.com/watch?v

In [21]:
import csv
title_truth_dict : dict = {}
print(title_dict)
f = open("FVC_text_queries.csv", 'r')
next(f)
for row in csv.reader(f):
    try:
        # print(row[1])
        vid_title = title_dict.get(row[1])
        title_truth_dict[vid_title] = row[2]
    except:
        print("Error")
    
print(title_truth_dict)

{'https://www.youtube.com/watch?v=Y7MJlmbnLQA': 'بنت سقطت في حمم بركانيه 🙀', 'https://www.youtube.com/watch?v=AEPvSo8bE2I': 'Michel & Sven - Der Tischdeckentrick - Teil 4 / The Table Cloth Magic Trick (Original)', 'https://www.youtube.com/watch?v=CXOU10WBogs': 'Backflip and almost hit by car!', 'https://www.youtube.com/watch?v=ZoHAf7VtIhA': 'Shark Attack - Cam Girl Bitten off Florida Coast', 'https://www.youtube.com/watch?v=RXF5yZZe8nQ': 'American Airline plane catches fire on runway', 'https://www.youtube.com/watch?v=fSbssO2DR7A': 'One Wing Landing', 'https://www.youtube.com/watch?v=PtWNknwUCE8': 'Reporter hit by stop sign during live tv.', 'https://www.youtube.com/watch?v=2YTntpldkhw': 'Kid from make a wish foundation gets tackled', 'https://www.youtube.com/watch?v=0NVUSOm8Mps': 'The Wrath of an Inflatable Raft', 'https://www.youtube.com/watch?v=F4jjR4vkwkM': 'Ski Trick Fail', 'https://www.youtube.com/watch?v=jGuDpD-Y-1s': 'girl falls through window', 'https://www.youtube.com/watch?v

In [22]:
# f = open("title_truth_dict.txt", 'w')
# json.dump(title_truth_dict, f)
# f.close()

In [23]:
import pandas as pd

In [24]:
print(video_captions_dict)

{'$250,000 car gets windshield SMASHED by kid on a skateboard!!!': ' a crowd of people walking down a street a man is driving a orange lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb lamb a person riding a skateboard down a street a street with a sign and a car parked on the side a man is standing in front of a car a man in an orange car driving down the street', "(Live and Breaking) Fire at Bangkok Nightclub - New Year's Eve": ' a group of people standing around a man a man is holding a cell phone a man in a white shirt is holding a knife a fire is seen in the sky above a building a man is standing in the dark a fire is seen in the sky above a building a car is seen in the dark at night a group of people walking down a street at night a woman is standing in the street with her arms outstretched a man is standing in the middle of a room a tornado is seen in this image from the video a man is holding a knife in his hand a man is seen in the dark as he walks through a crowd a

In [26]:
print(audio_captions_dict_cleaned)

{'$250,000 car gets windshield SMASHED by kid on a skateboard!!!': ' sorry Martin Shields milesplit', "(Live and Breaking) Fire at Bangkok Nightclub - New Year's Eve": '', '14-year-old girl thrown off ride at amusement park in Chongqing': '', '2011⧸3⧸11 東日本大震災 発生時(仙台市)  Earthquake in Sendai': " Brother Bear teriyaki bento you don't go over there after effects of sex what is 70 ml planet audio stereo Elsa Pataky Terence Timothy soccer", '2018-Bush cracks joke to Obama while Clinton makes a speech. Full video Link is in description.': '', '25 wild dogs against a cyclist': " yesterday yesterday Dora Dora Dora benefit Thomas Donna Donna Donna Donna roller bender daughter I'm never gonna go to yoga Miller Subaru top 10 top of the ladder Natasha Natasha when was the Holocaust the sports academy live on Lobos daughter logos meaning napa-sonoma tata Tata I'm gonna gonna take a little Thora Birch rewind 1 minute lotto tow truck Little Mermaid story Pretoria weather colour Bob Saget the box in m

In [32]:
# align features and labels
# going to need 4 dimensions per video
# title | audio | video | fake
# get title, fake from title_truth_dict; audio from audio_dict; video from video_dict
combined_info_dict : dict = {}

headers = ['title', 'truth', 'aud_cap', 'vid_cap']
titles = []
truths = []
audios = []
vid_caps = []

for title in title_truth_dict.keys():
    titles.append(title)
    truths.append(title_truth_dict[title])
    try:
        audios.append(audio_captions_dict_cleaned[title])
    except:
        audios.append("none")
    try:
        vid_caps.append(video_captions_dict[title])
    except:
        vid_caps.append("none")

combined_info_dict['title'] = titles
combined_info_dict['truth'] = truths
combined_info_dict['aud_caps'] = audios
combined_info_dict['vid_caps'] = vid_caps

print(combined_info_dict)

{'title': ['بنت سقطت في حمم بركانيه 🙀', 'Michel & Sven - Der Tischdeckentrick - Teil 4 / The Table Cloth Magic Trick (Original)', None, 'Backflip and almost hit by car!', 'Shark Attack - Cam Girl Bitten off Florida Coast', 'American Airline plane catches fire on runway', 'One Wing Landing', 'Reporter hit by stop sign during live tv.', 'Kid from make a wish foundation gets tackled', 'The Wrath of an Inflatable Raft', 'Ski Trick Fail', 'girl falls through window', 'Stormtrooper Falls Down Stairs On The Way To Star Wars Premiere', 'Efectos del huracán Irma en la isla de Antigua en la tarde del 5 de septiembre', 'امريكا تغرق - إعصار إرما المدمر والمشاهد المرعبه الاولي وكميه الدمار وسرعة الرياح-اعصار امريكا-harvy', 'True power of nature Hurricane  Irma UNLEASHED ON SHIPS + FREAKWAVES.', 'SOMEONE SHARED ME  *National Geographic channel has paid   $ 1 million for this daring video. 👌*', 'Tornado in Cape Town - South Africa - 6 June 2017 HD', 'Unbelievable: Strange Water Falls From Heaven in T

In [33]:
df = pd.DataFrame(combined_info_dict)

In [34]:
df.head()

,title,truth,aud_caps,vid_caps
0,بنت سقطت في حمم بركانيه 🙀,fake,,
1,Michel & Sven - Der Tischdeckentrick - Teil 4 ...,fake,none,none
2,None,real,none,none
3,Backflip and almost hit by car!,fake,i-ready no way talking,a man is walking down the street in the fall
4,Shark Attack - Cam Girl Bitten off Florida Coast,fake,show timing in Pune city evening Android auto,a boat is sailing in the ocean with a city in...


In [38]:
features = df.drop(["truth"], axis=1)
features.head()

,title,aud_caps,vid_caps
0,بنت سقطت في حمم بركانيه 🙀,,
1,Michel & Sven - Der Tischdeckentrick - Teil 4 ...,none,none
2,None,none,none
3,Backflip and almost hit by car!,i-ready no way talking,a man is walking down the street in the fall
4,Shark Attack - Cam Girl Bitten off Florida Coast,show timing in Pune city evening Android auto,a boat is sailing in the ocean with a city in...


In [40]:
labels = df["truth"]
print(labels.head())

0    fake
1    fake
2    real
3    fake
4    fake
Name: truth, dtype: object


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.15, random_state=42, shuffle=True
)

X_train.head()

,title,aud_caps,vid_caps
194,Video evidence shown during the trial of accus...,none,none
137,Attacks in Paris - During Bataclan Theatre att...,Peppa Pig good morning we have to talk you do...,a man is sitting on the ground in front of a ...
238,Woman carted off by Muslim horde into subway s...,,a man is seen in this image from the video a ...
75,Авиаудар по складу террористов в провинции РАККА,,
117,We got kicked out of a Delta airplane because ...,none,none


In [ ]:
history = model.fit(
    x = X_train, y = y_train,
    epochs=5, validation_split=0.05
)